<a href="https://colab.research.google.com/github/MrvZzZzZ/Lab1_AI_for_study/blob/main/wiki_parse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai
!pip install newspaper3k
!pip install openai mwclient mwparserfromhell tiktoken wikipedia-api

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 91.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.5 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=7832fb153308d18ffb53c32395a52da7e0bfb17f6861283eb7950edba760bb78
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=22d2df9a40558b4b621255102ef951e04b3d28b80f9be867015d11e2ff4ee160
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f16

In [12]:
from google.colab import userdata
from openai import OpenAI
import os
import getpass

GPT_MODEL = "gpt-3.5-turbo"

key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

base_url = "https://api.vsegpt.ru/v1"
os.environ["OPENAI_BASE_URL"] = base_url

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import mwparserfromhell  # Парсер для MediaWiki
import openai  # будем использовать для токинизации
import pandas as pd  # В DataFrame будем хранить базу знаний и результат токинизации базы знаний
import re  # для вырезания ссылок <ref> из статей Википедии
import tiktoken  # для подсчета токенов

In [4]:
!pip install trafilatura requests mwparserfromhell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 20.6 MB/s eta 0:00:00


In [5]:
import requests
import time
import mwparserfromhell
from typing import Set, List, Tuple
import re
import tiktoken
def get_category_pages_api(
    category_name: str,
    base_url: str = "https://ru.wikipedia.org",
    max_depth: int = 1,
    delay: float = 0.3
) -> Set[str]:
    API_ENDPOINT = f"{base_url}/w/api.php"
    category_name = category_name.strip()

    headers = {
        "User-Agent": "MyBot/1.0 (your@email.example)"
    }

    def recurse(cat: str, depth: int) -> Set[str]:
        titles = set()
        if depth < 0:
            return titles

        cmtitle = f"Категория:{cat}"

        params = {
            "action": "query",
            "list": "categorymembers",
            "cmtitle": cmtitle,
            "cmprop": "title|type|ns",
            "cmlimit": "max",
            "format": "json"
        }

        try:
            response = requests.get(API_ENDPOINT, params=params, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()

            if "error" in data:
                print(f"API error for '{cmtitle}': {data['error'].get('info', 'unknown')}")
                return titles

            members = data.get("query", {}).get("categorymembers", [])
            if not members and "continue" not in data:
                print(f"Категория '{cmtitle}' существует, но пуста или не содержит статей на этом уровне.")

            for m in members:
                title = m["title"]
                ns = m.get("ns", 0)
                page_type = m.get("type", "page")

                if ns == 0:
                    titles.add(title)
                elif page_type == "subcat" and depth > 0:
                    if title.startswith("Категория:"):
                        subcat = title[len("Категория:"):]
                    else:
                        subcat = title
                    titles.update(recurse(subcat, depth - 1))

            while "continue" in data:
                params.update(data["continue"])
                response = requests.get(API_ENDPOINT, params=params, headers=headers, timeout=10)
                data = response.json()
                members = data.get("query", {}).get("categorymembers", [])
                for m in members:
                    title = m["title"]
                    ns = m.get("ns", 0)
                    page_type = m.get("type", "page")
                    if ns == 0:
                        titles.add(title)
                    elif page_type == "subcat" and depth > 0:
                        subcat = title[len("Категория:"):] if title.startswith("Категория:") else title
                        titles.update(recurse(subcat, depth - 1))

        except Exception as e:
            print(f"Ошибка при обработке '{cmtitle}': {e}")
            return titles

        time.sleep(delay)
        return titles

    return recurse(category_name, max_depth)

def get_page_text(title: str, lang: str = "ru") -> str:
    """Получает wikitext статьи через action=raw."""
    url = f"https://{lang}.wikipedia.org/w/index.php"
    params = {
        "action": "raw",
        "title": title,
        "textonly": "1"
    }
    headers = {"User-Agent": "MyBot/1.0 (your@email.example)"}
    response = requests.get(url, params=params, headers=headers, timeout=10)
    response.raise_for_status()
    return response.text


def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode,
    parent_titles: List[str],
    sections_to_ignore: Set[str]
) -> List[Tuple[List[str], str]]:
    headings = section.filter_headings()
    if not headings:
        return []

    first_heading = headings[0]
    title_clean = first_heading.title.strip_code().strip()
    title_raw = str(first_heading)

    if title_clean in sections_to_ignore:
        return []

    titles = parent_titles + [title_raw]

    section_nodes = section.nodes
    content_nodes = []
    found_heading = False
    for node in section_nodes:
        if isinstance(node, mwparserfromhell.nodes.Heading):
            if not found_heading:
                found_heading = True
                continue
            else:
                break
        if found_heading:
            content_nodes.append(node)

    section_text = ''.join(str(n) for n in content_nodes).strip()

    current_level = first_heading.level
    results = [(titles, section_text)]

    for subsection in section.get_sections(levels=[current_level + 1]):
        results.extend(
            all_subsections_from_section(subsection, titles, sections_to_ignore)
        )
    return results

def all_subsections_from_title(
    title: str,
    sections_to_ignore: Set[str] = None,
    lang: str = "ru"
) -> List[Tuple[List[str], str]]:
    if sections_to_ignore is None:
        sections_to_ignore = {
            "См. также", "Примечания", "Литература", "Ссылки", "Внешние ссылки",
            "See also", "References", "Bibliography", "External links"
        }

    try:
        wikitext = get_page_text(title, lang=lang)
    except Exception as e:
        print(f"Не удалось загрузить «{title}»: {e}")
        return []

    parsed = mwparserfromhell.parse(wikitext)
    headings = parsed.filter_headings()

    if headings:
        first_heading_str = str(headings[0])
        full_text = str(parsed)
        summary = full_text.split(first_heading_str, 1)[0].strip()
    else:
        summary = str(parsed).strip()

    results = [([title], summary)]

    for section in parsed.get_sections(levels=[2]):
        results.extend(
            all_subsections_from_section(section, [title], sections_to_ignore)
        )
    return results

def clean_section(section: Tuple[List[str], str]) -> Tuple[List[str], str]:
    titles, text = section
    text = re.sub(r"<ref\b[^>]*/?>", "", text)
    text = re.sub(r"</ref\s*>", "", text)
    text = re.sub(r"\{\{(?:[SsEe]fn|note[Tt]ag|refn?|уточнить|источник|fact)\b[^}]*\}\}", "", text, flags=re.IGNORECASE)
    text = text.strip()
    return (titles, text)

def keep_section(section: Tuple[List[str], str]) -> bool:
    _, text = section
    words = text.split()
    return len(words) >= 3 and len(text) >= 20

In [8]:
key_articles = [
    "Великая Отечественная война",
    "Московская битва",
    "Сталинградская битва",
    "Курская битва",
    "Операция «Багратион»",
    "Блокада Ленинграда",
    "Операция «Уран»",
    "Операция «Кутузов»",
    "Операция «Румянцев»",
    "Ясско-Кишинёвская операция",
    "Начало Великой Отечественной войны",
    "Парад Победы",
    "День Победы",
    "Международная конференция в Ялте",
    "Потсдамская конференция",
    "Сталин, Иосиф Виссарионович",
    "Жуков, Георгий Константинович",
    "Рокоссовский, Константин Константинович",
    "Чуйков, Василий Иванович",
    "Василевский, Александр Михайлович",
    "Молотов, Вячеслав Михайлович",
    "Дорога жизни",
    "Трудовой фронт в СССР во время Великой Отечественной войны",
    "Эвакуация в СССР в годы Великой Отечественной войны",
    "Обращение Молотова по радио 22 июня 1941 года",
    "Речь Сталина 3 июля 1941 года",
    "Песни Великой Отечественной войны",
    "Берггольц, Ольга Фёдоровна",
    "Ленинградская филармония в годы блокады"
]

In [20]:
titles = get_category_pages_api(
    category_name="Великая Отечественная война",
    base_url="https://ru.wikipedia.org",
    max_depth=0,
    delay=0.8
)
# titles = set(key_articles)  # ← руками заданный мини-корпус
# print(f"Статей: {len(titles)} (ручной выбор)")
print(f"Собрано {len(titles)} статей.")

wikipedia_sections: List[Tuple[List[str], str]] = []

for i, title in enumerate(titles, 1):
    try:
        sections = all_subsections_from_title(title, lang="ru")
        wikipedia_sections.extend(sections)
    except Exception as e:
        print(f"Ошибка при обработке «{title}»: {e}")
    time.sleep(0.5)

print(f"\nВсего извлечено {len(wikipedia_sections)} секций из {len(titles)} статей.")

original_num = len(wikipedia_sections)
wikipedia_sections = [clean_section(ws) for ws in wikipedia_sections]
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]

print(f"Отфильтровано {original_num - len(wikipedia_sections)} секций, осталось {len(wikipedia_sections)}.")

print("\nПримеры оставшихся секций:")
for i, (path, text) in enumerate(wikipedia_sections[:5], 1):
    print(f"{i}. {' → '.join(path[-2:])}")
    print(f"   {text[:100].replace(chr(10), ' ').strip()}...")

Собрано 94 статей.

Всего извлечено 582 секций из 94 статей.
Отфильтровано 26 секций, осталось 556.

Примеры оставшихся секций:
1. Волжская рокада
   {{Железнодорожная линия}}  '''Во́лжская рока́да''' — [[Рокадная железная дорога|рокадная линия желез...
2. Волжская рокада → == Решение Госкомитета обороны ==
   Осенью 1941 года [[Оккупация территории СССР войсками Третьего рейха и его союзников|наступающие час...
3. Волжская рокада → == Изыскания и определение трассы ==
   Генерал-майор Фёдор Алексеевич Гвоздёвский немедленно выехал из Сталинграда по берегу Волги в сторон...
4. Волжская рокада → == Строительство ==
   17 марта 1942 года ГКО утвердил проект Волжской рокады.  А тем временем на трассе уже были проведены...
5. Волжская рокада → == Ввод в строй ==
   7 августа 1942 года, на 103-й день после начала земляных работ, первый поезд прошёл по участку «Илов...


In [22]:
print("Примеры собранных статей (первые 20):")
for t in sorted(titles):
    print(f"• {t}")

print(f"\nВсего: {len(titles)}")
print("\nЕсть ли нужные нам ключевые статьи?")
key_articles = [
    "Великая Отечественная война",
    "Московская битва",
    "Сталинградская битва",
    "Курская битва",
    "Операция «Багратион»",
    "Блокада Ленинграда",
    "Парад Победы",
    "Рокоссовский, Константин Константинович",
    "Чуйков, Василий Иванович",
    "Сталин, Иосиф Виссарионович"
]

found = [a for a in key_articles if any(a.lower() in t.lower() for t in titles)]
missing = [a for a in key_articles if not any(a.lower() in t.lower() for t in titles)]

print("Найдены:", found)
print("Отсутствуют:", missing)

Примеры собранных статей (первые 20):
• Агарковское движение
• Аксайская переправа
• Алтайский «Артек»
• Арктические конвои
• Арктические конвои (СССР)
• Бой под Зализницей
• Братский союз военнопленных
• Будь героем!
• Бытовые отряды
• Великая Отечественная война
• Великая Отечественная война (название)
• Военно-шефские комиссии
• Военнопленные в СССР во время Второй мировой войны
• Военный налог (1941—1945)
• Волжская рокада
• Встреча на Эльбе
• Всё для фронта! Всё для победы!
• Выставка образцов трофейного вооружения (Москва)
• Выступление Молотова по радио 22 июня 1941 года
• Выступление Сталина по радио 3 июля 1941 года
• Город первого салюта
• Городской комитет обороны
• Движение двухсотников
• Движение тысячников
• Депортация венгров и немцев Закарпатья
• Десять сталинских ударов
• Железнодорожная линия Астрахань — Кизляр
• Железнодорожная линия Сталинград — Владимировка
• Знак ранения
• Знамя Победы
• Ислам в годы Великой Отечественной войны
• Использование принудительного труд

In [14]:
# Функция подсчета токенов
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Функция разделения строк
def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Разделяет строку надвое с помощью разделителя (delimiter), пытаясь сбалансировать токены с каждой стороны."""

    # Делим строку на части по разделителю, по умолчанию \n - перенос строки
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # разделитель не найден
    elif len(chunks) == 2:
        return chunks  # нет необходимости искать промежуточную точку
    else:
        # Считаем токены
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        # Предварительное разделение по середине числа токенов
        best_diff = halfway
        # В цикле ищем какой из разделителей, будет ближе всего к best_diff
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        # Возвращаем левую и правую часть оптимально разделенной строки
        return [left, right]


# Функция обрезает строку до максимально разрешенного числа токенов
def truncated_string(
    string: str, # строка
    model: str, # модель
    max_tokens: int, # максимальное число разрешенных токенов
    print_warning: bool = True, # флаг вывода предупреждения
) -> str:
    """Обрезка строки до максимально разрешенного числа токенов."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    # Обрезаем строку и декодируем обратно
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Предупреждение: Строка обрезана с {len(encoded_string)} токенов до {max_tokens} токенов.")
    # Усеченная строка
    return truncated_string

# Функция делит секции статьи на части по максимальному числу токенов
def split_strings_from_subsection(
    subsection: tuple[list[str], str], # секции
    max_tokens: int = 1000, # максимальное число токенов
    model: str = GPT_MODEL, # модель
    max_recursion: int = 5, # максимальное число рекурсий
) -> list[str]:
    """
    Разделяет секции на список из частей секций, в каждой части не более max_tokens.
    Каждая часть представляет собой кортеж родительских заголовков [H1, H2, ...] и текста (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # Если длина соответствует допустимой, то вернет строку
    if num_tokens_in_string <= max_tokens:
        return [string]
    # если в результате рекурсия не удалось разделить строку, то просто усечем ее по числу токенов
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # иначе разделим пополам и выполним рекурсию
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]: # Пробуем использовать разделители от большего к меньшему (разрыв, абзац, точка)
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # если какая-либо половина пуста, повторяем попытку с более простым разделителем
                continue
            else:
                # применим рекурсию на каждой половине
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1, # уменьшаем максимальное число рекурсий
                    )
                    results.extend(half_strings)
                return results
    # иначе никакого разделения найдено не было, поэтому просто обрезаем строку (должно быть очень редко)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]


In [15]:
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)} секций Википедии поделены на {len(wikipedia_strings)} строк.")

Предупреждение: Строка обрезана с 1918 токенов до 1600 токенов.
Предупреждение: Строка обрезана с 1893 токенов до 1600 токенов.
407 секций Википедии поделены на 810 строк.


In [16]:
print(wikipedia_strings[0])

Молотов, Вячеслав Михайлович

{{←|Молотов}}
{{ФИО|Скрябин, Вячеслав Михайлович}}
{{Государственный деятель
| оригинал имени = Вячеслав Михайлович Скрябин
| изображение = Вячеслав Молотов (1947).jpg
| ширина = 250
| описание изображения = Министр иностранных дел СССР и член Политбюро ЦК ВКП (б) В. М. Молотов
| должность = Постоянный представитель СССР при [[Международное агентство по атомной энергии|МАГАТЭ]]
| флаг = Flag of USSR.svg
| флаг2 = International Atomic Energy Agency Logo.svg
| периодначало = 1960
| периодконец = 1963
| предшественник = [[Замятин, Леонид Митрофанович|Леонид Замятин]]
| преемник = [[Пономаренко, Пантелеймон Кондратьевич|Пантелеймон Пономаренко]]
| должность_2 = [[Список послов СССР и России в Монголии|Чрезвычайный и полномочный посол СССР в Монгольской Народной Республике]]
| флаг_2 = Flag of USSR.svg
| флаг2_2 = Flag of the Mongolian People's Republic (1945–1992).svg
| периодначало_2 = 31 августа 1957
| периодконец_2 = 3 июля 1960
| предшественник_2 = [[Писар

In [17]:
client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

def get_embedding(text, model="text-embedding-ada-002"):
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [19]:
import pandas as pd

df = pd.DataFrame({"text": wikipedia_strings})

df['embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

SAVE_PATH = "./Great World Patriotic War.csv"

df.to_csv(SAVE_PATH, index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       10 non-null     object
 1   embedding  10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes
